In [1]:
import json
import pandas as pd
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [4]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/usr/local/python/3.10.13/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [01:13<00:00, 12.96it/s]


In [6]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:26<00:00, 35.17it/s]


#### Hybrid search example

In [8]:
course = "data-engineering-zoomcamp"

In [9]:
query = 'I just discovered the course. Can I still join it?'

In [10]:
v_q = model.encode(query)

In [11]:
knn_query = {
    "field": "text_vector",
    "query_vector": v_q,
    "k": 5,
    "num_candidates": 10000,
    "boost": 0.5,
    "filter": {
        "term": {
            "course": course
        }
    }
}

In [12]:
keyword_query = {
    "bool": {
        "must": {
            "multi_match": {
                "query": query,
                "fields": ["question^3", "text", "section"],
                "type": "best_fields",
                "boost": 0.5,
            }
        },
        "filter": {
            "term": {
                "course": course
            }
        }
    }
}

In [13]:
response = es_client.search(
    index=index_name,
    #query=keyword_query,
    knn=knn_query,
    size=5
)

In [14]:
response["hits"]["hits"][0]

{'_index': 'course-questions',
 '_id': 'BvPhupEBWkM8z2dhEU8J',
 '_score': 0.37126788,
 '_source': {'text': "No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.",
  'section': 'General course-related questions',
  'question': 'Certificate - Can I follow the course in a self-paced mode and get a certificate?',
  'course': 'data-engineering-zoomcamp',
  'id': 'cb257ee5',
  'question_vector': [0.0030623162165284157,
   -0.0029858634807169437,
   -0.08672525733709335,
   0.024413973093032837,
   -0.05229020118713379,
   0.0012036776170134544,
   -0.09696734696626663,
   -0.05221624672412872,
   -0.08195184916257858,
   -0.04978093132376671,
   0.015911133959889412,
   0.04225786775350571,
   0.011777931824326515,
   0.035311635583639145,
   0.04309980571269989,
   

In [15]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [16]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [17]:
ground_truth[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

In [18]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)
    

In [19]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [20]:
def elastic_search_hybrid(field, query, vector, course):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question", "text", "section"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "course": course
                }
            }
        }
    }

    search_query = {
        "knn": knn_query,
        "query": keyword_query,
        "size": 5,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [21]:
def question_hybrid(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_hybrid('question_vector', question, v_q, course)

In [22]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [23]:
evaluate(ground_truth, question_hybrid)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [01:30<00:00, 51.25it/s]


{'hit_rate': 0.9241409120380376, 'mrr': 0.8483610690872416}

In [24]:
def text_hybrid(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_hybrid('text_vector', question, v_q, course)

In [25]:
evaluate(ground_truth, text_hybrid)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [01:22<00:00, 55.90it/s]


{'hit_rate': 0.923276421007132, 'mrr': 0.8461854333261294}

In [26]:
def question_text_hybrid(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_hybrid('question_text_vector', question, v_q, course)

evaluate(ground_truth, question_text_hybrid)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [01:22<00:00, 56.30it/s]


{'hit_rate': 0.9252215258266695, 'mrr': 0.8507492255601183}

ES knn on questions and answers: {'hit_rate': 0.9172249837907932, 'mrr': 0.824306606152295}

#### Reranking

To use the Reciprocal rank fusion (RRF) score we need to pull the docker image with a more recent version of Elasticsearch:

docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.9.0

In [27]:
def elastic_search_hybrid_rrf(field, query, vector, course):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question", "text", "section"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "course": course
                }
            }
        }
    }

    search_query = {
        "knn": knn_query,
        "query": keyword_query,
        "size": 5,
        "rank": {
            "rrf": {}
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [28]:
course = "data-engineering-zoomcamp"

In [29]:
query = 'I just discovered the course. Can I still join it?'

In [30]:
v_q = model.encode(query)

In [ ]:
elastic_search_hybrid_rrf('question_text_vector', query, v_q, course)

By default, RRF isn't available in a free-tier subscription. But you can try to use 30-day trial or upgrade the subscription plan.

#### RRF implementation

In [32]:
def compute_rrf(rank, k=60):
    """ Our own implementation of the relevance score """
    return 1 / (k + rank)

def elastic_search_hybrid_rrf(field, query, vector, course, k=60):
    knn_query = {
        "field": field,
        "query_vector": vector,
        "k": 10,
        "num_candidates": 10000,
        "boost": 0.5,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question", "text", "section"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
            "filter": {
                "term": {
                    "course": course
                }
            }
        }
    }

    knn_results = es_client.search(
        index=index_name, 
        body={
            "knn": knn_query, 
            "size": 10
        }
    )['hits']['hits']
    
    keyword_results = es_client.search(
        index=index_name, 
        body={
            "query": keyword_query, 
            "size": 10
        }
    )['hits']['hits']
    
    rrf_scores = {}
    # Calculate RRF using vector search results
    for rank, hit in enumerate(knn_results):
        doc_id = hit['_id']
        rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Adding keyword search result scores
    for rank, hit in enumerate(keyword_results):
        doc_id = hit['_id']
        if doc_id in rrf_scores:
            rrf_scores[doc_id] += compute_rrf(rank + 1, k)
        else:
            rrf_scores[doc_id] = compute_rrf(rank + 1, k)

    # Sort RRF scores in descending order
    reranked_docs = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Get top-K documents by the score
    final_results = []
    for doc_id, score in reranked_docs[:5]:
        doc = es_client.get(index=index_name, id=doc_id)
        final_results.append(doc['_source'])
    
    return final_results

In [33]:
def question_text_hybrid_rrf(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_hybrid_rrf('question_text_vector', question, v_q, course)

In [34]:
evaluate(ground_truth, question_text_hybrid_rrf)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [02:42<00:00, 28.49it/s]


{'hit_rate': 0.9524529933001945, 'mrr': 0.8748505150925734}

ES hybrid search scores: {'hit_rate': 0.9250054030689432, 'mrr': 0.8506231539514445}